# Practical 7: Interactive maps

In this final practial of the workshop we will create interactive maps. We will revisit some of the choropleth mapping techniques and we will explore some other mapping techniques, too. Interactive maps can be a great option for displaying and sharing data online. Viewers have the option to interact with your visualisation, making it more engaging.

We will continue to work with our example data from Southampton, although we will look at a few different datasets from before.

Learning objectives:
* create an interactive map with basemap and multiple layers
* save the output as a `.html` file
* display attribute information in pop-up windows
* create density map and know when to apply this technique


## Preparations for mapping

To make our interactive maps, we will be using `folium`. This package links with the `leaflet.js` library ([https://leafletjs.com/](https://leafletjs.com/)).

The documentation for `folium` can be found here: [https://python-visualization.github.io/folium/](https://python-visualization.github.io/folium/). We'll point a few key features as we go along, but be sure to pay attention to your CRS (again!). `folium` maps need inputs in latitude/longitude, so we'll be reprojecting from EPSG:27700. We will also need to convert our `GeoDataFrames` into GeoJSON formats.

In [ ]:
# load the packages
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

### Making a simple basemap

An overview map with a set of tiles for a basemap can be created with just a couple lines of code. 

We will supply the `.Map` with a `location` to tell it where to center the map initially, as well as an initial zoom level. Zoom levels are values from typically 0 to 18. Small numbers are "zoomed out" while higher numbers are "zoomed in".

In [ ]:
# simple basemap
# create an instance of a `Map`
m = folium.Map(location=[50.934358, -1.399073],  # latitude, longitude
               zoom_start=15
              )

# show the map
m

Try panning and zooming the map. Leaflet maps are very common on the internet now, so this should seem familiar to you.

### Changing the basemap tiles

By default our basempa is using OpenStreetMap tiles. There are few other sets of tiles available from Leaflet. Other tiles can be accessed if you have accounts with providers.

To change the basemap, set `tiles` parameter.

In [ ]:
# changing the basemap
m = folium.Map(location=[50.934358, -1.399073], 
               tiles='Stamen watercolor',  # artisitc basemap style
               zoom_start=15
              )

# show the map
m

We can also add another basemap to our `Map`. This step will not create a new map object, but add a layer as an option. We will also add a new control object to the map so that a user can toggle on/off the basemap of their choice.

Notice the layer control button in the upper-right of the map below

In [ ]:
# add basemap option
folium.TileLayer('stamentoner').add_to(m)

# add a control to select layers
folium.LayerControl().add_to(m)

m

### Load the data

This is great to be able to quickly create an interactive basemap, but we need to add in our own data. See this step as part of a large workflow - you can do various data cleaning and spatial analysis steps using `GeoPandas` and the other Python tools and then create sophisticated visualisations 

Let's load up the food point locations for Southampton.

In [ ]:
# load supermarkets
smkt = gpd.read_file('../data/soton_smkts.gpkg')
# convert from multipoint to single point
smkt = smkt.explode()
# project to WGS84
smkt = smkt.to_crs(4326)

smkt.head()

In [ ]:
# load fast food restaurants
ff = gpd.read_file('../data/soton_fastfood.gpkg')
# convert from multipoin to single point
ff = ff.explode()
# reproject to WGS84
ff = ff.to_crs(4326)

ff.head()

## Simple marker map

In order add these points to our map, we have to convert our DataFrames into GeoJSON format and then create a new layer of `Markers`. 

In [ ]:
# conversion
ff_gj = folium.features.GeoJson(ff, 
                                name='Fast Food'  # give the layer a name
                               )

In [ ]:
# create a fresh map
m = folium.Map(location=[50.934358, -1.399073],
               zoom_start=12,
               control_scale=True  # add a scale bar to the map
              )

# add the points
ff_gj.add_to(m)

# include control for turning on/off points
folium.LayerControl().add_to(m)

# show the map
m

### Multiple layers

We will now add another set of markers for our supermarkets, but we're going to do it differently to show how to create `Markers`. This will not convert the GeoDataFrame to GeoJSON, but it will access the coordinates in a loop across the features. This process will also let us easily set a custom icon colour for the supermarket layer.

In [ ]:
# create basemap
m = folium.Map(location=[50.934358, -1.399073],
               zoom_start=12,
               control_scale=True  # add a scale bar to the map
              )

# add the fast food points (default blue colour)
ff_gj.add_to(m)

# add the supermarket points the map
for idx, feature in smkt.iterrows():
    marker = folium.map.Marker([feature['geometry'].y, feature['geometry'].x], 
                               icon=folium.Icon(color='green')
                              ).add_to(m)

# show the results
m

### Clustering markers

Interactive maps can sometimes look a bit too cluttered with markers, especially when zoomed out further. One solution for this is to cluster the markers into groups that are visible when a user moves in closer. We'll revise our map to cluster the fast food points and add this as a new layer.

We will make use of one `folium`'s useful plugins: [`MarkerCluster`](https://github.com/python-visualization/folium/blob/master/examples/MarkerCluster.ipynb).

In [ ]:
from folium.plugins import MarkerCluster

In [ ]:
# start with a base map
m = folium.Map(location=[50.934358, -1.399073],
               zoom_start=12,
               control_scale=True
              )

In [ ]:
# create a set of locations from the fast food points
# lists of coordinate pairs in lat, long
locns = list(zip(ff.geometry.y, ff.geometry.x))

In [ ]:
# create the marker clusters
mc = MarkerCluster(locns)

# add to the map
mc.add_to(m)

# show the map
m

Try hovering over each cluster and then clicking. Or zooming in or out on the data.

## Heat maps

Sometimes our objective with a map is not to show individual features (like a set of points), but to highlight the broader spatial pattern. In the case of fast food restaurants, we may want a map to highlight areas with a greater density of sites. We can use a different type of map, one which is often called a *heat map*. 

Heat maps use a 2D kernel to express the density of point locations as a surface. Another plugin within `folium` provides easy access to this functionality.

#### Side note:

To create kernel density (i.e. heat maps) in a static map, we can use `seaborn`'s `kdeplot` ([https://seaborn.pydata.org/generated/seaborn.kdeplot.html](https://seaborn.pydata.org/generated/seaborn.kdeplot.html)).

In [ ]:
# load plugin
from folium.plugins import HeatMap

In [ ]:
# create a new, clean basemap
m = folium.Map(location=[50.934358, -1.399073],
               zoom_start=12,
               control_scale=True
              )

In [ ]:
# calculate a heat map
# use locations (see above)
hm = HeatMap(locns, 
             name='FF_heatmap'
            )

# add to map
hm.add_to(m)

# add control
folium.LayerControl().add_to(m)

m

There are a lot of restaurants in the city centre and Shirley High Street!

Interactive heat maps are scale (or zoom) dependent. Notice how the density changes as you zoom in and out. Try creating another heat map with fast food locations and alter the `radius` (default 25) and `blur` (default 15) options.

In [ ]:
# create a new heat map layer
# adjust radius=, and blur=


#

## Choropleth maps

We will return now to the common choropleth map that we introduced in the previous practicals on static maps. Since we're now building interactive maps, we can add a few new features to these, including pop-ups and actions when a user hovers over the data.

To make things a bit more interesting, we will also introduce a new dataset and review some previous processing steps on merging data.

The data we are using are the Index of Multiple Deprivation (IMD) which ranks each LSOA based on multiple criteria. We'll map the decile ranks. Values of 1 are among the most deprived areas. Values of 10 are among the least deprived 10% of areas in all of England. Find out more about these data here: [https://www.gov.uk/government/statistics/english-indices-of-deprivation-2019](https://www.gov.uk/government/statistics/english-indices-of-deprivation-2019).

In [ ]:
import pandas as pd

# load the LSOA boundary data
lsoa = gpd.read_file('../data/soton_lsoa_distance.gpkg')

# load IMD table
imd = pd.read_csv('../data/IMD2019_Index_of_Multiple_Deprivation.csv')
# rename columns for convenience
imd = imd.rename(columns={'LSOA code (2011)':'lsoa',
                          'Index of Multiple Deprivation (IMD) Decile':'imd_decile'
                         })
imd = imd[['lsoa','imd_decile']]

In [ ]:
imd.head()

In [ ]:
# create an inner join to keep only Southampton records
lsoa = lsoa.merge(imd, on='lsoa')

In [ ]:
lsoa.head()

In [ ]:
# make a simple static map of the index of multiple deprivation
lsoa.plot('imd_decile',
          categorical=True,
          legend=True,
          cmap='Reds',
          figsize=(15,10)
         )

plt.show()

Now if we want to create a similar map, but interactive using the tools from `folium`, we need to do a bit a preparation first.

#### Interactive format

### Colormaps and symbology

### Tooltips and pop-ups

## Saving the outputs